In [10]:
from openpyxl import load_workbook
import pandas as pd

class KnowledgeBase:
    def __init__(self):
        self.courses = {}

    def add_course(self, course):
        self.courses[course.code] = course

    def get_course(self, course_code):
        return self.courses.get(course_code)

    def load_courses_from_excel(self, file_path):
        workbook = load_workbook(file_path)
        sheet = workbook.active

        for row in sheet.iter_rows(min_row=2, values_only=True):
            Code, CourseName, CourseMap, prerequisite, Description, credit = row
            course = Course(Code, CourseName, CourseMap, prerequisite, Description, credit)
            self.add_course(course)

    def print_courses(self):
        for course in self.courses.values():
            print(f"Code: {course.code}, Name: {course.name}, Map: {course.map}, Prerequisite: {course.pre}, Description: {course.description}, credit: {course.credit}")

class Course:
    def __init__(self, Code, CourseName, CourseMap, prerequisite, description, credit):
        self.code = Code
        self.name = CourseName
        self.map = CourseMap
        self.pre = prerequisite
        self.description = description
        self.credit = credit

# Define the rules
def pass_logic_design_and_computer_architecture(student_courses):
    return "Logic Design" in student_courses and "Computer Architecture" in student_courses

def pass_embedded_system_with_minimum_grade(student_courses, grades):
    return "Embedded Systems" in student_courses and grades.get("Embedded Systems", 0) >= 60

def withdraw_course_before_week_13(course, week_number):
    return week_number <= 13

def take_required_courses_for_graduation(student_courses):
    required_courses = ["English 0", "English 1", "Human Rights"]
    return all(course in student_courses for course in required_courses)

def consecutive_semesters_below_threshold(gpa_history):
    return all(gpa < 1.00 for gpa in gpa_history[-3:])

def attendance_requirement_met(attendance_percentage):
    return attendance_percentage >= 75

def max_hours_for_improvement(duration, hours_retaken):
    if duration == 4:
        return hours_retaken <= 9
    elif duration == 5:
        return hours_retaken <= 12
    else:
        return False

def expelled_student_opportunity_completed_percentage(total_credit_hours, completed_credit_hours):
    return (completed_credit_hours / total_credit_hours) >= 0.8

# Read the Excel file and create KnowledgeBase
kb = KnowledgeBase()
kb.load_courses_from_excel("kbs3.xlsx")

# Function to add new data to Excel file
def add_data():
    file_path = "kbs3.xlsx"  # Update the file path if necessary
    new_data = {
        'Course Code': input("Enter Course Code: "),
        'Course Name': input("Enter Course Name: "),
        'Course Map': input("Enter Course Map: "),
        'Prerequisite': input("Enter Prerequisite: "),
        'Description': input("Enter Description: "),
        'credit': int(input("Enter credit: "))  
    }
    
    # Add new data to Excel file
    df = pd.DataFrame([new_data])
    with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
        df.to_excel(writer, index=False, header=False)
    
    # Update KnowledgeBase with the new course
    course = Course(new_data['Course Code'], new_data['Course Name'], new_data['Course Map'], new_data['Prerequisite'], new_data['Description'], new_data['credit'])
    kb.add_course(course)
    print("Data added successfully.")

# Main loop
while True:
    print("\nMENU:")
    print("1. Print Data")
    print("2. Add Data")
    print("3. Exit")
    choice = input("Enter your choice (1/2/3): ")
    
    if choice == '1':
        kb.print_courses()
    elif choice == '2':
        add_data()
    elif choice == '3':
        print("Exiting program.")
        break
    else:
        print("Invalid choice. Please enter 1, 2, or 3.")



MENU:
1. Print Data
2. Add Data
3. Exit
Enter your choice (1/2/3): 3
Exiting program.


In [11]:
from openpyxl import load_workbook
import pandas as pd
import tkinter as tk
from tkinter import messagebox

class AdvisingSystem:
    def __init__(self, knowledge_base):
        self.knowledge_base = knowledge_base

    def recommend_courses(self, gpa, passed_courses, failed_courses):
        recommended_courses = []

        # Recommendation based on GPA
        if gpa == 4.0:
            messagebox.showinfo("Recommendation", "Congratulations! Your GPA is 4.0. Aim for high distinction.")
            return recommended_courses
        elif gpa >= 3.5:
            elective_courses = [course for course in self.knowledge_base.courses.values() if "Elective" in course.map]
            if elective_courses:
                recommended_courses.append((elective_courses[0], f"Reason: This course is an elective and you have a GPA of {gpa} or higher."))
        elif gpa >= 3.0:
            messagebox.showinfo("Recommendation", "Your GPA is above 3.0. You are overload, You can register for more than 20 credit hours")
        elif gpa >= 2.0:
            messagebox.showinfo("Recommendation", "Your GPA is between 2.0 and 3.0. You have normal load, You can register for max 20 credit hours")
        else:
            messagebox.showinfo("Recommendation", "Your GPA is less than 2.0. You can only register for a half load (maximum 12 hours).")
            return recommended_courses
        # Recommendation based on failed courses
        for course_code in failed_courses:
            failed_course = self.knowledge_base.get_course(course_code.strip())
            if failed_course:
                recommended_courses.append((failed_course, "Reason: You have previously failed this course."))

        # Recommendation based on passed courses
        for course in self.knowledge_base.courses.values():
            if can_take_course(passed_courses, course) and course not in recommended_courses:
                recommended_courses.append((course, "Reason: You have passed the prerequisite courses."))

        if recommended_courses:
            recommendation_message = "Here are the courses you may consider along with the reasons:\n\n"
            for course, reason in recommended_courses:
                recommendation_message += f"Code: {course.code}, Name: {course.name}, Map: {course.map}, Prerequisite: {course.pre}, Description: {course.description}, Credit: {course.credit}\n{reason}\n\n"
            messagebox.showinfo("Recommendations", recommendation_message)
        else:
            messagebox.showinfo("Recommendations", "No recommendations at this time.")

    def recommend_passed_on_gpa(self, gpa_threshold):
        recommended_courses = []
        if gpa_threshold >= 3.5:
            for course in self.knowledge_base.courses.values():
                if "Elective" in course.map:
                    recommended_courses.append((course, f"Reason: This course is an elective and you have a GPA of {gpa_threshold} or higher."))
                    break  # Only recommend one elective course
        return recommended_courses



# Define the rules
def can_take_course(passed_courses, course):
    prerequisites = course.pre.split(", ")
    return all(prerequisite in passed_courses for prerequisite in prerequisites)

# Read the Excel file and create KnowledgeBase
kb = KnowledgeBase()
kb.load_courses_from_excel("kbs3.xlsx")

# GUI
def main():
    root = tk.Tk()
    root.title("Student Advising System")

    tk.Label(root, text="GPA:").pack()
    gpa_entry = tk.Entry(root)
    gpa_entry.pack()

    tk.Label(root, text="Passed Courses (comma-separated):").pack()
    passed_courses_entry = tk.Entry(root)
    passed_courses_entry.pack()

    tk.Label(root, text="Failed Courses (comma-separated):").pack()
    failed_courses_entry = tk.Entry(root)
    failed_courses_entry.pack()

    def get_recommendations():
        try:
            gpa = float(gpa_entry.get())
            passed_courses = passed_courses_entry.get().split(",")
            failed_courses = failed_courses_entry.get().split(",")

            advising_system = AdvisingSystem(kb)
            advising_system.recommend_courses(gpa, passed_courses, failed_courses)
        except ValueError:
            messagebox.showerror("Error", "Please enter a valid GPA.")

    tk.Button(root, text="Get Recommendations", command=get_recommendations).pack()

    root.mainloop()

if __name__ == "__main__":
    main()
